In [37]:
import json
import pandas as pd

In [2]:
# Requires extra packages.
!pip install tqdm boto3 requests regex sentencepiece sacremoses
!pip install tokenizers
!pip install huggingface-hub
import pickle
import cloudpickle as cp
import math
import sklearn.metrics # Area Under the ROC calculations.
import matplotlib.pylab as plt # Plotting
from urllib.request import urlopen
import torch
#Downloads a tokenizer that will automatically convert words to indices in a big dictionary.
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

Using cache found in /home/clark/.cache/torch/hub/huggingface_pytorch-transformers_master


In [3]:
tweet = "% Sorry losers and haters, but my I.Q. is one of the highest"
indexed_tokens = tokenizer.encode(tweet, padding='max_length', add_special_tokens=True) 
# Create transition matrix as sparse matrix to save memory.
print(indexed_tokens)
n = tokenizer.vocab_size
print(n)

[101, 1003, 3374, 23160, 1998, 5223, 2869, 1010, 2021, 2026, 1045, 1012, 1053, 1012, 2003, 2028, 1997, 1996, 3284, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [4]:
# print(tokenizer.decode(indexed_tokens, skip_special_tokens=True))

In [5]:
from pathlib import Path
paths = [str(x) for x in Path('./filesplit/').glob('*.json')]
paths[:10]

['filesplit/reddit99917556.json',
 'filesplit/reddit998757.json',
 'filesplit/reddit997973.json',
 'filesplit/reddit99922195.json',
 'filesplit/reddit99909782.json',
 'filesplit/reddit99912979.json',
 'filesplit/reddit99905760.json',
 'filesplit/reddit99911182.json',
 'filesplit/reddit99900043.json',
 'filesplit/reddit99903226.json']

In [6]:
len(paths)

34119

In [100]:
# Python program to read
# json file
 
count = 0
list1 = []

matches = ["xbox", "Xbox", "XBOX"]
# Opening JSON file
for i in range(100):
    f = open(paths[i], 'r')
    data = [json.loads(line) for line in f]
    for item in data:
        if any(x in item["subreddit"] for x in matches):
            list1.append(item)


In [101]:
len(list1)

88

In [102]:
list1[2]["subreddit"]

'DayzXbox'

In [103]:
list2 = []
for item in list1:
    if item["selftext"] != "[removed]":
        list2.append(item)

In [104]:
len(list2)

77

In [105]:
list2[0]["selftext"]

'As the title says, I have not owned an Xbox since the Xbox360, and after seeing the exclusives coming to this console, I had to get my hands on one. Last week Walmart had a drop, and I scored a series X, this console is fantastic, the ultimate gamepass is a game changer. I own a ps5 as well, and I am truly excited for this generation of gaming.\n\nLooking forward to joining the Xbox community online, I don’t have many friends on there, hopefully change that soon. The Bethesda exclusives, Halo, Forza, and gamepass ultimate made me realize I was being one dimensional on my console preferences. \n\nI must say Gears 5 looks fantastic on my 65” OLED. \n\nTo everyone still trying for a console, be patient, you will get one I promise!'

In [106]:
list1[6]["selftext"]

''

In [107]:
len(list1)

88

In [46]:
# for item in data:
#     if "made" in item:
#         print("yes")

print(data[102]["selftext"])

hey, i've just started getting into tech wear and finding a distinctive style but im a Line/prep cook who doesnt make that much money . im also a plus size woman (2-3x )i want to know everyones favorite budget friendly plus size finds or shops. 

thank you so much i feel alil silly asking but i hope yall can help


In [47]:
if "hey" in data[102]["selftext"]:
    print("yes")

yes


In [165]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [118]:
df = pd.read_csv("./data/reddit.csv")

In [119]:
X = df.clean_comment

In [120]:
Y = df.category

In [111]:
x_train, x_test, y_train, y_test = train_test_split(
...     X, Y, test_size=0.33, random_state=42)

In [112]:
x_train.head(10)

33960    can’ hire someone hack series and delete there...
28142                          zelda reminds sae san from 
14657    chill man you are much indian eat beef and hum...
11762    based india pede love meeting other people who...
31903                                  they banned reddit 
29710                                                  NaN
5937     even does not problem for the bjp since law an...
26933     the twitter there request for shirt for yrs o...
1261     amsa tweets signing off positions the malaysia...
33791                                      looks familiar 
Name: clean_comment, dtype: object

In [115]:
sample = x_train.iloc[0]

In [121]:
len(X)

37249

In [123]:
X.iloc[1]

'buddhism has very much lot compatible with christianity especially considering that sin and suffering are almost the same thing suffering caused wanting things shouldn want going about getting things the wrong way christian this would mean wanting things that don coincide with god will and wanting things that coincide but without the aid jesus buddhism could also seen proof god all mighty will and omnipotence certainly christians are lucky have one such christ there side but what about everyone else well many christians believe god grace salvation and buddhism god way showing grace upon others would also help study the things jesus said and see how buddha has made similar claims such rich man getting into heaven joke basically advocating that should rid ourselves material possessions fact distinctly remembered jesus making someone cry because that someone asked what achieve salvation and jesus replied with live like buddhist very very roughly translated also point out that buddha rare

In [127]:
from tqdm.notebook import tqdm

In [140]:
import math

In [155]:
P = X.isnull()

In [158]:
Z = X[P == False]

In [159]:
Z[:10]

0     family mormon have never tried explain them t...
1    buddhism has very much lot compatible with chr...
2    seriously don say thing first all they won get...
3    what you have learned yours and only yours wha...
4    for your own benefit you may want read living ...
5    you should all sit down together and watch the...
6     was teens when discovered zen meditation was ...
7                             jesus was zen meets jew 
8    there are two varieties christians dogmatic th...
9    dont worry about trying explain yourself just ...
Name: clean_comment, dtype: object

In [160]:
X = Z

In [260]:
Y = Y[P == False]

In [261]:
Y[:10]

0    1
1    1
2   -1
3    0
4    1
5   -1
6    1
7    0
8   -1
9    1
Name: category, dtype: int64

In [161]:
# stuff = "hello there"
stuff = X.iloc[413]
print(stuff)
if stuff != :
    print("hes")
print(type(stuff))
indexed_tokens = tokenizer.encode(stuff, padding='max_length', add_special_tokens=True) 
# indexed_tokens

SyntaxError: invalid syntax (<ipython-input-161-f39246825199>, line 4)

In [299]:
X[1]

'buddhism has very much lot compatible with christianity especially considering that sin and suffering are almost the same thing suffering caused wanting things shouldn want going about getting things the wrong way christian this would mean wanting things that don coincide with god will and wanting things that coincide but without the aid jesus buddhism could also seen proof god all mighty will and omnipotence certainly christians are lucky have one such christ there side but what about everyone else well many christians believe god grace salvation and buddhism god way showing grace upon others would also help study the things jesus said and see how buddha has made similar claims such rich man getting into heaven joke basically advocating that should rid ourselves material possessions fact distinctly remembered jesus making someone cry because that someone asked what achieve salvation and jesus replied with live like buddhist very very roughly translated also point out that buddha rare

In [333]:
allSamples = []
with tqdm(total=len(X)) as pbar:
    for i in range(len(X)):
        if i not in listRemove:
            indexed_tokens = tokenizer.encode(X.iloc[i], padding='max_length', add_special_tokens=True) 
            allSamples.append(np.array(indexed_tokens))
            pbar.update(1)

  0%|          | 0/37149 [00:00<?, ?it/s]

In [335]:
len(Y)

37149

In [336]:
Y = Y.drop(Y.index[listRemove])

In [337]:
len(Y)

37016

In [320]:
len(allSamples[914])

519

In [323]:
listRemove = []
count = 0
for item in allSamples:
    if len(item) > 512:
        print("index: {}, length = {}".format(count, len(item)))
        listRemove.append(count)
    count += 1

index: 914, length = 519
index: 1248, length = 818
index: 1320, length = 581
index: 1352, length = 968
index: 1597, length = 516
index: 1678, length = 571
index: 1680, length = 1166
index: 1683, length = 1113
index: 2370, length = 695
index: 2794, length = 780
index: 3187, length = 583
index: 3464, length = 827
index: 4178, length = 1123
index: 4222, length = 973
index: 4366, length = 1258
index: 4633, length = 724
index: 4634, length = 1028
index: 4668, length = 531
index: 4801, length = 545
index: 5071, length = 537
index: 5172, length = 557
index: 5355, length = 648
index: 5733, length = 558
index: 5741, length = 875
index: 5858, length = 530
index: 6006, length = 1250
index: 6899, length = 762
index: 6915, length = 778
index: 7001, length = 585
index: 7216, length = 1224
index: 7274, length = 542
index: 7291, length = 705
index: 7334, length = 623
index: 7616, length = 643
index: 7993, length = 690
index: 8056, length = 593
index: 8224, length = 889
index: 8238, length = 517
index:

In [324]:
len(listRemove)

133

In [326]:
len(allSamples)

37149

In [325]:
type(Y)

pandas.core.series.Series

In [338]:
df = pd.DataFrame(allSamples)

In [294]:
allSamples[0]

array([  101,  2155, 15111,  2031,  2196,  2699,  4863,  2068,  2027,
        2145,  6237, 14909,  2013,  2051,  2051,  2066,  2070,  2785,
        4326,  6492,  9690,  2027,  2031,  2272, 19837,  2005,  1996,
       11752,  5475,  2791,  1041, 16211,  3490, 16383,  9920,  1998,
       15398,  2031,  2764,  2035,  1996,  2477, 11388, 12011,   102,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [339]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,101,2155,15111,2031,2196,2699,4863,2068,2027,2145,...,0,0,0,0,0,0,0,0,0,0
1,101,11388,2038,2200,2172,2843,11892,2007,7988,2926,...,0,0,0,0,0,0,0,0,0,0
2,101,5667,2123,2360,2518,2034,2035,2027,2180,2131,...,0,0,0,0,0,0,0,0,0,0
3,101,2054,2017,2031,4342,6737,1998,2069,6737,2054,...,0,0,0,0,0,0,0,0,0,0
4,101,2005,2115,2219,5770,2017,2089,2215,3191,2542,...,0,0,0,0,0,0,0,0,0,0
5,101,2017,2323,2035,4133,2091,2362,1998,3422,1996,...,0,0,0,0,0,0,0,0,0,0
6,101,2001,13496,2043,3603,16729,13804,2001,2059,6151,...,0,0,0,0,0,0,0,0,0,0
7,101,4441,2001,16729,6010,16522,102,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,101,2045,2024,2048,9903,8135,3899,12644,2008,23120,...,0,0,0,0,0,0,0,0,0,0
9,101,2123,2102,4737,2055,2667,4863,4426,2074,19960,...,0,0,0,0,0,0,0,0,0,0


In [329]:
listRemove[:10]

[914, 1248, 1320, 1352, 1597, 1678, 1680, 1683, 2370, 2794]

In [330]:
df2 = df.drop(df.index[listRemove])

In [331]:
len(df2)

37016

In [332]:
df2.head(10)

,0,1,2,3,4,5,6,7,8,9,...,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079
0,101,2155,15111,2031,2196,2699,4863,2068,2027,2145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101,11388,2038,2200,2172,2843,11892,2007,7988,2926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101,5667,2123,2360,2518,2034,2035,2027,2180,2131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101,2054,2017,2031,4342,6737,1998,2069,6737,2054,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101,2005,2115,2219,5770,2017,2089,2215,3191,2542,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,101,2017,2323,2035,4133,2091,2362,1998,3422,1996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,101,2001,13496,2043,3603,16729,13804,2001,2059,6151,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,101,4441,2001,16729,6010,16522,102,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,101,2045,2024,2048,9903,8135,3899,12644,2008,23120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,101,2123,2102,4737,2055,2667,4863,4426,2074,19960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
import numpy as np

In [190]:
allSamples = np.array(allSamples)

<ipython-input-190-b2115c1310cd>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  allSamples = np.array(allSamples)


In [257]:
type(allSamples)

list

In [259]:
allSamples[0]

[101,
 2155,
 15111,
 2031,
 2196,
 2699,
 4863,
 2068,
 2027,
 2145,
 6237,
 14909,
 2013,
 2051,
 2051,
 2066,
 2070,
 2785,
 4326,
 6492,
 9690,
 2027,
 2031,
 2272,
 19837,
 2005,
 1996,
 11752,
 5475,
 2791,
 1041,
 16211,
 3490,
 16383,
 9920,
 1998,
 15398,
 2031,
 2764,
 2035,
 1996,
 2477,
 11388,
 12011,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0

In [116]:
indexed_tokens = tokenizer.encode(sample, padding='max_length', add_special_tokens=True) 

In [117]:
indexed_tokens

[101,
 2064,
 1521,
 10887,
 2619,
 20578,
 2186,
 1998,
 3972,
 12870,
 2045,
 3149,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [287]:
x_train, x_test, y_train, y_test = train_test_split(
...     allSamples, Y, test_size=0.33, random_state=42)

In [218]:
x_train = np.array(x_train)

<ipython-input-218-3d41624a13f9>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array(x_train)


In [269]:
type(x_train)

list

In [288]:
x_train[:10]

[array([  101,  2714,  2466,  2001,  1996,  2028,  2040,  2109,  2954,
         2007,  4013,  9779,  2361,  2814,  2077,  2297,  2027,  2020,
         8040, 23606,  7476, 13675, 16585, 16498,  2043, 17712,  7317,
        20784,  2001,  2730,  2001, 17772,  2008, 16913,  2072,  2097,
        13848,  5853,  2122,  8840, 10698,  2229,  2110,  2097,  2552,
         4189,  4047, 10644,  2964,  2059, 12434,  2231,  7917,  6240,
         2005,  2070, 17136,  2782,  4415,  2110,  2001,  2085, 17678,
        16098,  3436,  7560,  9189,  2050,  2150,  3424, 16913,  2072,
         6880,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [221]:
num = 2
print(x_train[num])
print(y_train.iloc[num])

[  101 10166  4171  3477  2121  5029  2131 21183 21758  3435  2005  2477
  2066  2023   102     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [234]:
x_train.shape

(24889,)

In [236]:
x_train[:10]

array([array([  101,  2714,  2466,  2001,  1996,  2028,  2040,  2109,  2954,
               2007,  4013,  9779,  2361,  2814,  2077,  2297,  2027,  2020,
               8040, 23606,  7476, 13675, 16585, 16498,  2043, 17712,  7317,
              20784,  2001,  2730,  2001, 17772,  2008, 16913,  2072,  2097,
              13848,  5853,  2122,  8840, 10698,  2229,  2110,  2097,  2552,
               4189,  4047, 10644,  2964,  2059, 12434,  2231,  7917,  6240,
               2005,  2070, 17136,  2782,  4415,  2110,  2001,  2085, 17678,
              16098,  3436,  7560,  9189,  2050,  2150,  3424, 16913,  2072,
               6880,   102,     0,     0,     0,     0,     0,     0,     0,
                  0,     0,     0,     0,     0,     0,     0,     0,     0,
                  0,     0,     0,     0,     0,     0,     0,     0,     0,
                  0,     0,     0,     0,     0,     0,     0,     0,     0,
                  0,     0,     0,     0,     0,     0,     0,     0,     0,

In [235]:
y_train.shape

(24889,)

In [290]:
x_train[0].shape

(512,)

In [243]:
p_train = x_train.reshape(-1, 24889, 512)

ValueError: cannot reshape array of size 24889 into shape (24889,512)

In [ ]:
x_train = x_train.reshape(-1, 1, 9)
x_test  = x_test.reshape(-1, 1, 9)
y_train = y_train.reshape(-1, 1, 5)
y_test = y_test.reshape(-1, 1, 5)

In [266]:
allSamples[:10]

[[101,
  2155,
  15111,
  2031,
  2196,
  2699,
  4863,
  2068,
  2027,
  2145,
  6237,
  14909,
  2013,
  2051,
  2051,
  2066,
  2070,
  2785,
  4326,
  6492,
  9690,
  2027,
  2031,
  2272,
  19837,
  2005,
  1996,
  11752,
  5475,
  2791,
  1041,
  16211,
  3490,
  16383,
  9920,
  1998,
  15398,
  2031,
  2764,
  2035,
  1996,
  2477,
  11388,
  12011,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [282]:
from sklearn.preprocessing import MinMaxScaler

In [285]:
x_train[0]

[101,
 2714,
 2466,
 2001,
 1996,
 2028,
 2040,
 2109,
 2954,
 2007,
 4013,
 9779,
 2361,
 2814,
 2077,
 2297,
 2027,
 2020,
 8040,
 23606,
 7476,
 13675,
 16585,
 16498,
 2043,
 17712,
 7317,
 20784,
 2001,
 2730,
 2001,
 17772,
 2008,
 16913,
 2072,
 2097,
 13848,
 5853,
 2122,
 8840,
 10698,
 2229,
 2110,
 2097,
 2552,
 4189,
 4047,
 10644,
 2964,
 2059,
 12434,
 2231,
 7917,
 6240,
 2005,
 2070,
 17136,
 2782,
 4415,
 2110,
 2001,
 2085,
 17678,
 16098,
 3436,
 7560,
 9189,
 2050,
 2150,
 3424,
 16913,
 2072,
 6880,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [291]:
Xscaler = MinMaxScaler(feature_range=(0, 1)) # scale so that all the X data will range from 0 to 1
Xscaler.fit(x_train[0])

ValueError: Expected 2D array, got 1D array instead:
array=[  101.  2714.  2466.  2001.  1996.  2028.  2040.  2109.  2954.  2007.
  4013.  9779.  2361.  2814.  2077.  2297.  2027.  2020.  8040. 23606.
  7476. 13675. 16585. 16498.  2043. 17712.  7317. 20784.  2001.  2730.
  2001. 17772.  2008. 16913.  2072.  2097. 13848.  5853.  2122.  8840.
 10698.  2229.  2110.  2097.  2552.  4189.  4047. 10644.  2964.  2059.
 12434.  2231.  7917.  6240.  2005.  2070. 17136.  2782.  4415.  2110.
  2001.  2085. 17678. 16098.  3436.  7560.  9189.  2050.  2150.  3424.
 16913.  2072.  6880.   102.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.    

In [276]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [351]:
len(x_train)

24800

In [360]:
type(x_train)

pandas.core.frame.DataFrame

In [361]:
generator = TimeseriesGenerator(x_train.to_numpy(), y_train.to_numpy(), length=25, batch_size=100)

In [362]:
type(generator)

keras.preprocessing.sequence.TimeseriesGenerator

In [367]:
generator[0]

(array([[[  101, 12063,  2123, ...,     0,     0,     0],
         [  101,  2064,  4339, ...,     0,     0,     0],
         [  101,  6135,  5993, ...,     0,     0,     0],
         ...,
         [  101,  2008,  2339, ...,     0,     0,     0],
         [  101,  2245, 26618, ...,     0,     0,     0],
         [  101,  2852,  3527, ...,     0,     0,     0]],
 
        [[  101,  2064,  4339, ...,     0,     0,     0],
         [  101,  6135,  5993, ...,     0,     0,     0],
         [  101,  2070,  6971, ...,     0,     0,     0],
         ...,
         [  101,  2245, 26618, ...,     0,     0,     0],
         [  101,  2852,  3527, ...,     0,     0,     0],
         [  101,  2138,  1996, ...,     0,     0,     0]],
 
        [[  101,  6135,  5993, ...,     0,     0,     0],
         [  101,  2070,  6971, ...,     0,     0,     0],
         [  101,  7760, 18471, ...,     0,     0,     0],
         ...,
         [  101,  2852,  3527, ...,     0,     0,     0],
         [  101,  2138, 

In [340]:
x_train, x_test, y_train, y_test = train_test_split(
...     df, Y, test_size=0.33, random_state=42)

In [371]:
print(generator[0][0].shape)

(100, 25, 512)


In [341]:
x_train.head(10)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
11323,101,12063,2123,2130,102,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10479,101,2064,4339,8822,5698,20624,9276,3343,3513,2007,...,0,0,0,0,0,0,0,0,0,0
321,101,6135,5993,2021,2054,2055,4926,13047,26781,13046,...,0,0,0,0,0,0,0,0,0,0
22303,101,2070,6971,2144,2111,2356,2005,2068,9152,14852,...,0,0,0,0,0,0,0,0,0,0
11544,101,7760,18471,102,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27510,101,1998,8398,2097,4604,2035,2068,2067,102,0,...,0,0,0,0,0,0,0,0,0,0
10275,101,2045,23411,4559,2634,24954,2097,2663,10476,12361,...,0,0,0,0,0,0,0,0,0,0
4291,101,26471,24954,2064,6616,1040,23274,3372,11998,11190,...,0,0,0,0,0,0,0,0,0,0
10701,101,2200,4415,2562,2122,2048,7832,4237,2113,2177,...,0,0,0,0,0,0,0,0,0,0
36465,101,2925,102,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [373]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM#, CuDNNLSTM


# mnist = tf.keras.datasets.mnist  # mnist is a dataset of 28x28 images of handwritten digits and their labels
# (x_train, y_train),(x_test, y_test) = mnist.load_data()  # unpacks images to x_train/x_test and labels to y_train/y_test

# x_train = x_train/255.0
# x_test = x_test/255.0

# print(x_train.shape)
# print(x_train[0].shape)

model = Sequential()

# IF you are running with a GPU, try out the CuDNNLSTM layer type instead (don't pass an activation, tanh is required)
model.add(LSTM(512, input_shape=(25,512), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))



model.add(Dense(1, activation='softmax'))



opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='mse',
    optimizer='adam',
    metrics=['accuracy'],
)

model.fit_generator(generator,
          epochs=3)

<ipython-input-373-c7a6b7c3dce1>:42: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator,


Epoch 1/3
248/248 [==============================] - 47s 185ms/step - loss: 1.2501 - accuracy: 0.4232
Epoch 2/3
248/248 [==============================] - 46s 185ms/step - loss: 1.2501 - accuracy: 0.4232
Epoch 3/3
248/248 [==============================] - 46s 185ms/step - loss: 1.2501 - accuracy: 0.4232
